# Cryptocurrency Tweets Analysis

**Group 7** - Smrithy Dharma Sankaran, David Marquardt, Beau Muster, Laura Yuan

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tweepy

In [2]:
# Reading datasets
tweets = pd.read_csv('/Users/laurayuan/Desktop/School/Masters/Spring_2021/Analytics_for_Finance/Final Project/crypto_tweets.csv')
prices = pd.read_csv('/Users/laurayuan/Desktop/School/Masters/Spring_2021/Analytics_for_Finance/Final Project/crypto_prices.csv')

/Users/laurayuan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Cleaning the dataset

In [3]:
# Getting columns we want
tweets = tweets[['date','tweet']].dropna()
tweets.head()

,date,tweet
0,2013-03-29,Bitcoin: The Cyberpunk Cryptocurrency
1,2013-04-15,Bitcoin Isn't the Only Cryptocurrency in Town via
2,2013-04-14,", a ""cryptocurrency"", went on a tear last week..."
3,2013-04-18,I'm going to make my OWN crypto-currency and e...
4,2013-05-09,"Your momma's cryptocurrency is so virtual, she..."


#### Extracting Sentiment from Tweets

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm.auto import tqdm
from tqdm import tqdm
import string
tqdm.pandas()

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

/Users/laurayuan/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/laurayuan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/laurayuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def get_tokens(text):
    res = []
    words = word_tokenize(text)
    for word in words:
        if word not in stop_words and word not in string.punctuation:
           res.append(PorterStemmer().stem(word)) 
    return res

In [6]:
tweets['tokens'] = tweets.tweet.progress_apply(lambda x: get_tokens(x))
tweets.head()

100%|██████████| 824901/824901 [09:28<00:00, 1450.11it/s]


,date,tweet,tokens
0,2013-03-29,Bitcoin: The Cyberpunk Cryptocurrency,"[bitcoin, the, cyberpunk, cryptocurr]"
1,2013-04-15,Bitcoin Isn't the Only Cryptocurrency in Town via,"[bitcoin, Is, n't, onli, cryptocurr, town, via]"
2,2013-04-14,", a ""cryptocurrency"", went on a tear last week...","[``, cryptocurr, '', went, tear, last, week, r..."
3,2013-04-18,I'm going to make my OWN crypto-currency and e...,"[I, 'm, go, make, own, crypto-curr, everyon, ...."
4,2013-05-09,"Your momma's cryptocurrency is so virtual, she...","[your, momma, 's, cryptocurr, virtual, think, ..."


In [7]:
with open('/Users/laurayuan/Desktop/School/Masters/Spring_2021/NLP/Harvard_Inquirer_Dictionary.txt', 'r') as dt_txt:
    HIDict = dt_txt.readlines()
HIDict = HIDict[1:]

In [8]:
#Extract all the lines that contain the Pos tag

poswords = [j for j in HIDict if "Pos" in j]  #using a list comprehension
poswords = [j.split()[0] for j in poswords]
poswords = [j.split("#")[0] for j in poswords]
poswords = set(PorterStemmer().stem(j.lower()) for j in poswords)

#Extract all the lines that contain the Neg tag
negwords = [j for j in HIDict if "Neg" in j]  #using a list comprehension
negwords = [j.split()[0] for j in negwords]
negwords = [j.split("#")[0] for j in negwords]
negwords = set(PorterStemmer().stem(j.lower()) for j in negwords)

In [9]:
def get_pos(tokens):
    res = []
    for t in tokens:
        if t in poswords:
            res.append(t)
    return res    

def get_neg(tokens):
    res = []
    for t in tokens:
        if t in negwords:
            res.append(t)
    return res 

In [10]:
tweets['pos']= tweets.tokens.apply(lambda x: get_pos(x))
tweets['neg']= tweets.tokens.apply(lambda x: get_neg(x))
tweets.head()

,date,tweet,tokens,pos,neg
0,2013-03-29,Bitcoin: The Cyberpunk Cryptocurrency,"[bitcoin, the, cyberpunk, cryptocurr]",[],[]
1,2013-04-15,Bitcoin Isn't the Only Cryptocurrency in Town via,"[bitcoin, Is, n't, onli, cryptocurr, town, via]",[],[]
2,2013-04-14,", a ""cryptocurrency"", went on a tear last week...","[``, cryptocurr, '', went, tear, last, week, r...",[close],"[tear, close]"
3,2013-04-18,I'm going to make my OWN crypto-currency and e...,"[I, 'm, go, make, own, crypto-curr, everyon, ....","[make, join]","[make, except]"
4,2013-05-09,"Your momma's cryptocurrency is so virtual, she...","[your, momma, 's, cryptocurr, virtual, think, ...",[your],[]


In [11]:
tweets['pos_count'] = tweets.pos.apply(lambda x: len(x))
tweets['neg_count'] = tweets.neg.apply(lambda x: len(x))
tweets.head()

,date,tweet,tokens,pos,neg,pos_count,neg_count
0,2013-03-29,Bitcoin: The Cyberpunk Cryptocurrency,"[bitcoin, the, cyberpunk, cryptocurr]",[],[],0,0
1,2013-04-15,Bitcoin Isn't the Only Cryptocurrency in Town via,"[bitcoin, Is, n't, onli, cryptocurr, town, via]",[],[],0,0
2,2013-04-14,", a ""cryptocurrency"", went on a tear last week...","[``, cryptocurr, '', went, tear, last, week, r...",[close],"[tear, close]",1,2
3,2013-04-18,I'm going to make my OWN crypto-currency and e...,"[I, 'm, go, make, own, crypto-curr, everyon, ....","[make, join]","[make, except]",2,2
4,2013-05-09,"Your momma's cryptocurrency is so virtual, she...","[your, momma, 's, cryptocurr, virtual, think, ...",[your],[],1,0


In [12]:
tweets = tweets.drop(columns = ['tokens','pos','neg'])
tweets['pos'] = tweets['pos_count'] / (tweets['pos_count'] + tweets['neg_count'])
tweets['neg'] = tweets['neg_count'] / (tweets['pos_count'] + tweets['neg_count'])

In [13]:
tweets = tweets.dropna()
tweets = tweets.drop(columns = ['pos_count','neg_count'])
tweets.head()

,date,tweet,pos,neg
2,2013-04-14,", a ""cryptocurrency"", went on a tear last week...",0.333333,0.666667
3,2013-04-18,I'm going to make my OWN crypto-currency and e...,0.500000,0.500000
4,2013-05-09,"Your momma's cryptocurrency is so virtual, she...",1.000000,0.000000
5,2013-05-09,"Your momma's cryptocurrency is so virtual, whe...",0.333333,0.666667
6,2013-05-14,Venture capital firm sends a signal to Bitcoin...,0.666667,0.333333


Cleaning prices:

In [14]:
prices = prices[['Name','Date','High','Low','Open','Close','Volume']]
prices.head()

,Name,Date,High,Low,Open,Close,Volume
0,Bitcoin,2013-04-29,147.488007,134.000000,134.44400,144.539993,0.0
1,Litecoin,2013-04-29,4.573600,4.225640,4.36676,4.383900,0.0
2,Bitcoin,2013-04-30,146.929993,134.050003,144.00000,139.000000,0.0
3,Litecoin,2013-04-30,4.572380,4.168960,4.40352,4.296490,0.0
4,Bitcoin,2013-05-01,139.889999,107.720001,139.00000,116.989998,0.0


In [15]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 721617 entries, 2 to 824903
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    721617 non-null  object 
 1   tweet   721617 non-null  object 
 2   pos     721617 non-null  float64
 3   neg     721617 non-null  float64
dtypes: float64(2), object(2)
memory usage: 27.5+ MB


In [16]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13049 entries, 0 to 13048
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Name    13049 non-null  object 
 1   Date    13049 non-null  object 
 2   High    13049 non-null  float64
 3   Low     13049 non-null  float64
 4   Open    13049 non-null  float64
 5   Close   13049 non-null  float64
 6   Volume  9080 non-null   float64
dtypes: float64(5), object(2)
memory usage: 713.7+ KB


### Filtering Sentiment for each Cryptocurrency

* Bitcoin

In [17]:
bitcoin_sent = tweets[(tweets['tweet'].str.contains('bitcoin')) |(tweets['tweet'].str.contains('Bitcoin'))]
bitcoin_sent.head()

,date,tweet,pos,neg
5,2013-05-09,"Your momma's cryptocurrency is so virtual, whe...",0.333333,0.666667
6,2013-05-14,Venture capital firm sends a signal to Bitcoin...,0.666667,0.333333
7,2013-06-28,Fascinating insights. MT : The bitcoin/cryptoc...,1.000000,0.000000
8,2013-08-12,Crypto-currency for NSA leaker: Snowden fund a...,1.000000,0.000000
9,2013-10-30,"Bitcoin’s radical days are over, says . Here’s...",0.000000,1.000000


In [18]:
bitcoin_sent = bitcoin_sent.groupby('date', as_index = False).agg({'pos':'mean','neg':'mean'})
bitcoin_sent['Name'] = 'Bitcoin'
bitcoin_sent.head()

,date,pos,neg,Name
0,2013-01-04,0.750000,0.250000,Bitcoin
1,2013-01-07,0.666667,0.333333,Bitcoin
2,2013-02-26,1.000000,0.000000,Bitcoin
3,2013-03-11,0.666667,0.333333,Bitcoin
4,2013-04-12,0.750000,0.250000,Bitcoin


* Binance

In [19]:
binance_sent = tweets[(tweets['tweet'].str.contains('binance')) |(tweets['tweet'].str.contains('Binance'))]

In [20]:
binance_sent = binance_sent.groupby('date', as_index = False).agg({'pos':'sum','neg':'sum'})
binance_sent['Name'] = 'Binance'
binance_sent.head()

,date,pos,neg,Name
0,2017-08-17,1.000000,0.000000,Binance
1,2017-08-26,4.000000,0.000000,Binance
2,2017-09-28,0.500000,0.500000,Binance
3,2017-10-06,1.000000,0.000000,Binance
4,2017-11-13,0.666667,0.333333,Binance


* Cardano

In [21]:
cardano_sent = tweets[(tweets['tweet'].str.contains('cardano')) |(tweets['tweet'].str.contains('Cardano'))]

In [22]:
cardano_sent = binance_sent.groupby('date', as_index = False).agg({'pos':'mean','neg':'mean'})
cardano_sent['Name'] = 'Cardano'
cardano_sent.head()

,date,pos,neg,Name
0,2017-08-17,1.000000,0.000000,Cardano
1,2017-08-26,4.000000,0.000000,Cardano
2,2017-09-28,0.500000,0.500000,Cardano
3,2017-10-06,1.000000,0.000000,Cardano
4,2017-11-13,0.666667,0.333333,Cardano


* Ethereum

In [23]:
ethereum_sent = tweets[(tweets['tweet'].str.contains('ethereum')) |(tweets['tweet'].str.contains('Ethereum'))]

In [24]:
ethereum_sent = binance_sent.groupby('date', as_index = False).agg({'pos':'mean','neg':'mean'})
ethereum_sent['Name'] = 'Ethereum'
ethereum_sent.head()

,date,pos,neg,Name
0,2017-08-17,1.000000,0.000000,Ethereum
1,2017-08-26,4.000000,0.000000,Ethereum
2,2017-09-28,0.500000,0.500000,Ethereum
3,2017-10-06,1.000000,0.000000,Ethereum
4,2017-11-13,0.666667,0.333333,Ethereum


* Litecoin

In [25]:
litecoin_sent = tweets[(tweets['tweet'].str.contains('litecoin')) |(tweets['tweet'].str.contains('Litecoin'))]

In [26]:
litecoin_sent = binance_sent.groupby('date', as_index = False).agg({'pos':'mean','neg':'mean'})
litecoin_sent['Name'] = 'Litecoin'
litecoin_sent.head()

,date,pos,neg,Name
0,2017-08-17,1.000000,0.000000,Litecoin
1,2017-08-26,4.000000,0.000000,Litecoin
2,2017-09-28,0.500000,0.500000,Litecoin
3,2017-10-06,1.000000,0.000000,Litecoin
4,2017-11-13,0.666667,0.333333,Litecoin


* XRP

In [27]:
xrp_sent = tweets[(tweets['tweet'].str.contains('xrp')) |(tweets['tweet'].str.contains('XRP'))]

In [28]:
xrp_sent = binance_sent.groupby('date', as_index = False).agg({'pos':'mean','neg':'mean'})
xrp_sent['Name'] = 'XRP'
xrp_sent.head()

,date,pos,neg,Name
0,2017-08-17,1.000000,0.000000,XRP
1,2017-08-26,4.000000,0.000000,XRP
2,2017-09-28,0.500000,0.500000,XRP
3,2017-10-06,1.000000,0.000000,XRP
4,2017-11-13,0.666667,0.333333,XRP


Merging sentiment analysis data:

In [29]:
frames = [bitcoin_sent, binance_sent, litecoin_sent, xrp_sent, ethereum_sent, cardano_sent]
crypto_sent = pd.concat(frames)
crypto_sent

,date,pos,neg,Name
0,2013-01-04,0.750000,0.250000,Bitcoin
1,2013-01-07,0.666667,0.333333,Bitcoin
2,2013-02-26,1.000000,0.000000,Bitcoin
3,2013-03-11,0.666667,0.333333,Bitcoin
4,2013-04-12,0.750000,0.250000,Bitcoin
...,...,...,...,...
742,2021-02-06,5.800000,0.200000,Cardano
743,2021-02-07,3.680952,3.319048,Cardano
744,2021-02-08,8.196429,0.803571,Cardano
745,2021-02-09,7.400000,1.600000,Cardano


### Data Exploration

In [30]:
tweets.describe()

,pos,neg
count,721617.000000,721617.000000
mean,0.685677,0.314323
std,0.313603,0.313603
min,0.000000,0.000000
25%,0.500000,0.000000
50%,0.727273,0.272727
75%,1.000000,0.500000
max,1.000000,1.000000


In [31]:
prices.describe()

,High,Low,Open,Close,Volume
count,13049.000000,13049.000000,13049.000000,13049.000000,9.080000e+03
mean,1175.796468,1124.220343,1150.530146,1157.069524,3.406050e+09
std,3310.143739,3073.633087,3195.777638,3215.526918,9.634364e+09
min,0.000000,0.000000,0.000000,0.003082,0.000000e+00
25%,2.967670,2.774110,2.873670,2.943040,7.943528e+06
50%,117.340000,115.740000,116.500000,116.850000,1.810494e+08
75%,679.728027,644.393005,664.796997,679.585999,2.059595e+09
max,57533.389325,51015.764554,57532.738864,55888.133682,3.509679e+11


In [32]:
# Gold & S&P exploration
sp_gold = prices[(prices['Name'] == 'Gold') | (prices['Name'] == 'S&P')]

In [33]:
sp_gold.groupby('Name').agg({'High':'mean', 'Low':'mean','Open':'mean', 'Close':'mean'}).sort_values(by = 'Name')

,High,Low,Open,Close
Name,,,,
Gold,128.691684,127.582482,128.153078,128.140767
S&P,2428.350716,2403.588387,2416.530762,2439.198211


In [34]:
# Cryptocurrencies exploration
cryptos_only = prices[(prices['Name'] != 'S&P') & (prices['Name'] != 'Gold')]

In [35]:
cryptos_only.groupby('Name').agg({'High':'mean', 'Low':'mean','Open':'mean', 'Close':'mean', 'Volume':'mean'}).sort_values(by = 'Name')

,High,Low,Open,Close,Volume
Name,,,,,
Binance,19.715108,17.866523,18.714476,18.979276,2.968918e+08
Bitcoin,4892.842241,4600.863853,4749.021436,4768.826575,9.201904e+09
Cardano,0.143449,0.126698,0.135877,0.136248,4.631058e+08
Ethereum,265.482446,244.386440,255.723179,256.010335,5.588557e+09
Litecoin,42.921996,39.247936,41.230862,41.200325,1.116044e+09
XRP,0.207515,0.186994,0.197485,0.197636,9.540319e+08


### Merging Tweets with Prices

In [36]:
# Merging
data = prices.merge(crypto_sent, how = 'left', left_on = ['Name', 'Date'], right_on = ['Name','date'])
data = data.drop(columns = 'date')
data = data.dropna()
data.head()

,Name,Date,High,Low,Open,Close,Volume,pos,neg
16,Bitcoin,2013-05-09,113.459999,109.260002,113.199997,112.669998,0.0,0.333333,0.666667
22,Bitcoin,2013-05-14,119.800003,110.250000,117.980003,111.500000,0.0,0.666667,0.333333
88,Bitcoin,2013-06-28,101.737000,92.333000,101.737000,94.649200,0.0,1.000000,0.000000
155,Bitcoin,2013-08-12,108.000000,103.500000,105.000000,106.636002,0.0,1.000000,0.000000
191,Bitcoin,2013-08-28,127.320000,122.620003,126.480003,122.620003,0.0,0.666667,0.333333


In [37]:
# Imbalanced data
data.groupby('Name')['Name'].count()

Name
Binance      543
Bitcoin     1666
Cardano      541
Ethereum     543
Litecoin     543
XRP          543
Name: Name, dtype: int64

In [38]:
# Finding dates where there are all 6 cryptos to fix imbalanced data
data = data.groupby('Date').filter(lambda x : x['Date'].count() == 6)
data.head()

,Name,Date,High,Low,Open,Close,Volume,pos,neg
3962,Bitcoin,2017-10-06,4413.270020,4320.529785,4324.459961,4370.810059,1.069940e+09,0.590541,0.409459
3963,Litecoin,2017-10-06,52.603100,51.530499,51.636002,52.120098,5.935590e+07,1.000000,0.000000
3964,Ethereum,2017-10-06,308.839996,294.947998,295.154999,308.588013,3.186640e+08,1.000000,0.000000
3965,Binance,2017-10-06,1.713580,1.534240,1.677310,1.579910,6.076260e+06,1.000000,0.000000
3966,Cardano,2017-10-06,0.021542,0.018360,0.021359,0.018539,7.780710e+06,1.000000,0.000000


In [40]:
# Check if still imbalanced
data.groupby('Name')['Name'].count()

Name
Binance     541
Bitcoin     541
Cardano     541
Ethereum    541
Litecoin    541
XRP         541
Name: Name, dtype: int64

In [41]:
# Export new data to csv
data.to_csv(r'prices_with_sentiment.csv', index=False)